In [1]:
import numpy as np
import qiskit.pulse as pulse
from qiskit_dynamics.pulse.pulse_simulator import PulseSimulator

In [3]:
from qiskit_ibm_provider import IBMProvider
#IBMProvider.save_account(token='e138ea6b754f60ffd002db17bcfc3aaf8eb7fd35f51c832aa247c8ffa6ca261b9ca3eb487d3a976b631469141fc693fdb6d37284560395483c47b27dec4a5e42')

provider = IBMProvider()
ibm_backend = provider.get_backend('ibmq_lima')

In [4]:
backend = PulseSimulator.from_backend(ibm_backend, subsystem_list=[0, 1])

In [5]:
from qiskit.pulse import library

amp = 1
sigma = 10
num_samples = 128
#%%
gauss = pulse.library.Gaussian(num_samples, amp, sigma,
                              name="Parametric Gauss")
gauss.draw()


with pulse.build() as schedule:
    pulse.play(gauss, backend.drive_channel(0))
    pulse.play(gauss, backend.drive_channel(1))
    pulse.play(gauss, backend.control_channel([0,1])[0])
    pulse.play(gauss, backend.control_channel([1,0])[0])

schedule.draw()
y0 = np.zeros(backend.solver.model.dim)
y0[0] = 1
t_span = np.array([0, num_samples * backend.solver._dt])
result = backend.run(schedule, y0=y0, t_span=t_span)
#%%
result

Result(backend_name='Pulse Simulator of ibmq_lima', backend_version='0.1', qobj_id='', job_id='b57a56ef-6f62-487f-b68d-1914c9b15ea6', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(unitary=[[ 1.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 1.00000000e+00+3.05266938e-12j  7.71727140e-07-4.06818693e-06j
   3.45267698e-11+3.05566593e-11j ... -1.57033634e-12+1.36744000e-11j
  -6.47632805e-18-1.54265430e-16j  4.77184814e-22+3.17358382e-22j]
 [ 1.00000000e+00+5.59332349e-11j  3.97288011e-06-8.18452302e-06j
  -2.16148012e-11-1.05029476e-10j ...  7.73970122e-11+1.54483939e-10j
   9.58354088e-16+2.46953333e-15j  2.82936235e-20-6.64772617e-21j]
 ...
 [ 7.65068905e-02+4.95191607e-02j  9.37432768e-02-2.76400991e-01j
   5.77561417e-03-8.50136233e-03j ...  3.51877809e-02-1.56592302e-03j
   3.

In [6]:
result.results[0]

ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(unitary=[[ 1.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 1.00000000e+00+3.05266938e-12j  7.71727140e-07-4.06818693e-06j
   3.45267698e-11+3.05566593e-11j ... -1.57033634e-12+1.36744000e-11j
  -6.47632805e-18-1.54265430e-16j  4.77184814e-22+3.17358382e-22j]
 [ 1.00000000e+00+5.59332349e-11j  3.97288011e-06-8.18452302e-06j
  -2.16148012e-11-1.05029476e-10j ...  7.73970122e-11+1.54483939e-10j
   9.58354088e-16+2.46953333e-15j  2.82936235e-20-6.64772617e-21j]
 ...
 [ 7.65068905e-02+4.95191607e-02j  9.37432768e-02-2.76400991e-01j
   5.77561417e-03-8.50136233e-03j ...  3.51877809e-02-1.56592302e-03j
   3.96514681e-02-5.24227565e-02j  1.60810844e-03-1.91647891e-03j]
 [ 7.65071482e-02+4.95159721e-02j  9.68024328e-02-2.77249787e-01j
   7.72463713e-03-1.87228583e

In [7]:
backend.solver._hamiltonian_channels

['d0', 'd1', 'u0', 'u1', 'u2', 'u3']